<a href="https://colab.research.google.com/github/seonmia/MRC_KoBigBird_KoELECTRA/blob/main/add_data_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easydict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import json
from pprint import pprint
import os
import random
import math
import csv
import json
from tqdm.notebook import tqdm
from easydict import EasyDict as edict
from statistics import mean
from typing import List, Tuple, Dict, Any

import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
from typing import List, Tuple, Dict, Any
import json
import random
import copy

class KoMRC:
    def __init__(self, data, indices: List[Tuple[int, int, int]]):
        self._data = data
        self._indices = indices

    # Json을 불러오는 메소드
    @classmethod
    def load(cls, file_path: str):
        with open(file_path, 'r', encoding='utf-8') as fd:
            data = json.load(fd)

        indices = []
        for d_id, document in enumerate(data['data']):
            for p_id, paragraph in enumerate(document['paragraphs']):
                for q_id, _ in enumerate(paragraph['qas']):
                    indices.append((d_id, p_id, q_id))
        
        return cls(data, indices)

##################################################################
    # 데이터 셋을 잘라내는 메소드
    @classmethod
    def split(cls, dataset, train_raio: float=.8,test_ratio: float=.5, seed=42):
        indices = list(dataset._indices)
        random.seed(seed)
        random.shuffle(indices)
        train_indices = indices[:int(len(indices) * train_raio)]
        split_indices = indices[int(len(indices) * train_raio):]
        eval_indices = split_indices[:int(len(split_indices) * test_ratio)]
        test_indices = split_indices[int(len(split_indices) * test_ratio):]

        return cls(dataset._data, train_indices), cls(dataset._data, eval_indices), cls(dataset._data, test_indices)
################################################################################
    # 데이터 셋을 잘라내는 메소드
    @classmethod
    def split(cls, dataset, eval_ratio: float=.1, seed=42):
        indices = list(dataset._indices)
        random.seed(seed)
        random.shuffle(indices)
        train_indices = indices[int(len(indices) * eval_ratio):]
        eval_indices = indices[:int(len(indices) * eval_ratio)]

        return cls(dataset._data, train_indices), cls(dataset._data, eval_indices)

    @classmethod
    def extraction(cls, dataset, data_number: int, save=True):
        '''
        첫 번째 인자 : KoMRC의 인스턴스
        두 번째 인자 : 추출할 data 개수 (단, data단위로 추출, indices단위가 아님)
        반환값 : KoMRC의 인스턴스
        '''
        if data_number > len(dataset):
            raise Exception("입력한 인자가 데이터셋 크기보다 큽니다.")
        else:
            data = copy.deepcopy(dataset._data)
            data['data'] = random.sample(data['data'], data_number)
            indices = []
            for d_id, document in enumerate(data['data']):
                for p_id, paragraph in enumerate(document['paragraphs']):
                    for q_id, _ in enumerate(paragraph['qas']):
                        indices.append((d_id, p_id, q_id))
            if save:
                with open("cut_result.json", "w") as write_file:
                    json.dump(data, write_file, indent=4)
        return cls(data, indices)

    def __add__(self, other) :
        data = copy.deepcopy(self._data)
        data['data'] = self._data['data'] + other._data['data']
        indices = []
        for d_id, document in enumerate(data['data']):
            for p_id, paragraph in enumerate(document['paragraphs']):
                for q_id, _ in enumerate(paragraph['qas']):
                    indices.append((d_id, p_id, q_id))
        return KoMRC(data, indices)

    

    def __getitem__(self, index: int) -> Dict[str, Any]:
        d_id, p_id, q_id = self._indices[index]
        paragraph = self._data['data'][d_id]['paragraphs'][p_id]

        context = paragraph['context']
        qa = paragraph['qas'][q_id]

        # 원래 코드
        guid = qa['guid']
        question = qa['question']
        answers = qa['answers']

        #guid = qa['guid'] ### 수정 ### 
        #question = qa['question']
        #   'answer_start' : qa['answers']['answer_start']}


        return {
            'guid': guid,
            'context': context,
            'question': question,
            'answers': answers
        }

    def __len__(self) -> int:
        return len(self._indices)

    

위의 KoMRC 클래스 코드에서 add함수 추가했고, guid, question, answers부분은 기존의 베이스 코드로 바꿨습니다.

아래의 코드는 예시코드입니다

In [ ]:
dataset1 = KoMRC.load('train.json') # train json 불러오고
dataset2 = KoMRC.load('TL_span_extraction2.json') # 사전에 형식을 train.json 파일과 똑같이 만들어 놓은 TL_span_extraction2.json 파일을 불러오고
dataset3 = dataset1 + dataset2 # 두 개를 합친다면
type(dataset1), type(dataset2), type(dataset3) # 위 3개의 인스턴스는 같은 클래스입니다.

(__main__.KoMRC, __main__.KoMRC, __main__.KoMRC)

In [ ]:
len(dataset1), len(dataset2), len(dataset3) # dataset3은 dataset1과 dataset2가 합쳐진 것입니다.

(12037, 223936, 235973)

In [ ]:
pprint(dataset1[0]) # dataset1의 데이터를 들여다보면

{'answers': [{'answer_start': 478, 'text': '한 달가량'},
             {'answer_start': 478, 'text': '한 달'}],
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 '
            '전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 '
            '호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 '
            '하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 '
            '내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 '
            '줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 '
            '장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 '
            '시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 '
            '전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, '
            '강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 '
            '내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 '
            '것으로 예상돼 거리 응원에는 지

In [ ]:
pprint(dataset2[0]) # dataset2의 데이터를 들여다보면

{'answers': [{'answer_start': 4,
              'clue_start': None,
              'clue_text': None,
              'options': None,
              'text': '서부권역 다목적 체육관'}],
 'context': '익산시 서부권역 다목적 체육관이 내달 개관을 앞두고 막바지 개관 준비가 한창이다. \n'
            '\n'
            '시에 따르면 시민들의 건강증진과 삶의 질 향상을 위한 생활체육 인프라 확충 등을 위해 지난 2019년 착공된 서부권역 '
            '다목적 체육관이 이달 말까지 시설물 준공을 완료하고 다음달부터 본격적인 개관 운영에 들어간다. \n'
            '\n'
            '전문 체육시설인 서부권역 다목적 체육관은 지하 1층과 지상 3층, 연면적 4719㎡ 규모로 총 사업비 '
            '124억5000만원이 투입됐다. \n'
            '\n'
            '수영장 6개 레인과 어린이풀장, 다목적실(에어로빅, 요가 등), 체력단련실, 어르신들을 위한 물리ㆍ재활 운동실 등이 '
            '들어서며, 수영, 헬스, 요가, 에어로빅, 아쿠아로빅 등 총 5가지 프로그램이 운영될 예정이다. \n'
            '\n'
            '첫 번째로 개관되는 시설은 헬스장이다. \n'
            '\n'
            '다음달 중순부터 회원 신청이 가능하며 체육관에 방문해 신청하면 된다. \n'
            '\n'
            '아울러 시는 코로나19 예방을 위해 소독작업은 물론 체육관 운영시간과 참여 인원을 축소해 운영하는 방안을 계획하고 '
            '있다. \n'
            '\n'
            '체육관 전체 소독은 하루 3회로 먼저 개관되는 헬스장의

In [ ]:
pprint(dataset3[-1]) # dataset3의 데이터를 들여다보면

{'answers': [{'answer_start': 202,
              'clue_start': None,
              'clue_text': None,
              'options': None,
              'text': '6명'}],
 'context': '광주대학교(총장 김혁종)는 지난 9일 학교 행정관 회의실에서 장기근속자 공로패 수여식 및 스승의날 유공 교원 '
            '교육부장관 표창 전수식을 개최했다.\n'
            '10일 광주대에 따르면 학교법인 호심학원 창설 51주년·광주대학교 개교 41주년을 기념해 진행된 이번 장기근속자 '
            '공로패 수여 대상자는 30년 근속 이봉규 인테리어디자인학과 교수와 박우미 패션·주일리디자인학부 교수 등 6명이다.\n'
            '이날 제40회 스승의날 유공 교원 교육부장관 표창 전수식도 함께 열렸다. 교육부장관 표창은 올바른 교육관과 투철한 '
            '사명감으로 사도를 실천하고 헌신적인 노력을 통해 교육 발전에 이바지한 교원에게 수여된다.\n'
            '김덕현 보건행정학부 교수와 임형택 청소년상담·평생교육학과 교수가 인재 양성 분야에서 수상의 영예를 안았다.\n'
            '김 교수는 연구와 실무경험으로 대학과 지역 산업계의 유대를 강화하고 이를 바탕으로 인재 양성에 기여한 공로를 '
            '인정받았다.\n'
            '또 임 교수는 다년간의 청소년 교육 분야 연구를 바탕으로 산학연계를 통한 맞춤형 교육과 후학양성에 노력한 점 등이 '
            '높은 평가를 받았다.',
 'guid': 4961424,
 'question': '광주대학교로부터 얼마나 되는 교원이 공로패를 전달받게 된 거야'}


In [ ]:
train, eval = KoMRC.split(dataset3, 0.1) # split도 해봅시다

In [ ]:
type(train), type(eval) # 둘다 KoMRC의 인스턴스

(__main__.KoMRC, __main__.KoMRC)

In [ ]:
len(train), len(eval) # 길이를 보니 잘 나누졌음

(212376, 23597)

#### 데이터 추출

In [ ]:
extraction_dataset = KoMRC.extraction(dataset3, 100) # data 100개 추출

In [ ]:
type(extraction_dataset)

__main__.KoMRC

In [ ]:
len(extraction_dataset)  # 100이 아닌 191인 이유: data는 100개지만, 각 데이터당 질문이 2개일 수 있기 때문.
# 즉, indeces기준 100개가 아닌 데이터 100개

191